# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [283]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Generate the mathematical model

In [335]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [336]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Food and Beverage Production Planning

**Sets:**

*  $T = \{1, 2, ..., 12\}$: Set of time periods.
*  $F = \{1, 2\}$: Set of food products.
*  $B = \{1, 2, 3\}$: Set of beverage products.

**Parameters:**

*  $FC_{it}$: Fixed cost of producing product $i \in F \cup B$ in period $t \in T$ (from "fixed_cost_production.csv").
*  $VCprod_{it}$: Variable cost of producing one unit of product $i \in F \cup B$ in period $t \in T$ (from "variable_cost_production.csv").
*  $VCstore_{it}$: Variable cost of storing one unit of product $i \in F \cup B$ in period $t \in T$ (from "variable_cost_storage.csv").
*  $D_{it}$: Demand for product $i \in F \cup B$ in period $t \in T$ (from "demand.csv").
*  $R_{it}$: Revenue per unit of product $i \in F \cup B$ sold in period $t \in T$ (from "revenue.csv").
*  $SC_i$: Storage capacity for product $i \in F \cup B$: 
    *  $SC_1 = 580$ (Food 1)
    *  $SC_2 = 687$ (Food 2)
    *  $SC_3 = 599$ (Beverage 1)
    *  $SC_4 = 788$ (Beverage 2)
    *  $SC_5 = 294$ (Beverage 3)
*  $PC_i$: Production capacity for product $i \in F \cup B$:
    *  $PC_1 = 1080$ (Food 1)
    *  $PC_2 = 908$ (Food 2)
    *  $PC_3 = 408$ (Beverage 1)
    *  $PC_4 = 1000$ (Beverage 2)
    *  $PC_5 = 403$ (Beverage 3)

**Decision Variables:**

*  $X_{it}$: Quantity of product $i \in F \cup B$ produced in period $t \in T$.
*  $S_{it}$: Quantity of product $i \in F \cup B$ stored at the end of period $t \in T$.
*  $Y_{it}$: Quantity of product $i \in F \cup B$ sold in period $t \in T$.

**Objective Function:**

Maximize total profit over the planning horizon:

$ \text{Maximize } Z = \sum_{t \in T} \sum_{i \in F \cup B} (R_{it}Y_{it} - VCprod_{it}X_{it} - VCstore_{it}S_{it} - FC_{it} \cdot \mathbb{1} \{X_{it} > 0\})$

where $\mathbb{1} \{X_{it} > 0\}$ is an indicator function that equals 1 if $X_{it} > 0$ and 0 otherwise.

**Constraints:**

1. **Production Capacity:**
    *  $X_{it} \le PC_i$ for all $i \in F \cup B$, $t \in T$.

2. **Storage Capacity:**
    *  $S_{it} \le SC_i$ for all $i \in F \cup B$, $t \in T$.

3. **Inventory Balance:**
    *  $S_{i(t-1)} + X_{it} - Y_{it} = S_{it}$ for all $i \in F \cup B$, $t \in T$, where $S_{i0} = 0$.

4. **Demand Satisfaction (Maximum Sales):**
    *  $Y_{it} \le D_{it}$ for all $i \in F \cup B$, $t \in T$.

5. **Non-negativity:**
    *  $X_{it}, S_{it}, Y_{it} \ge 0$ for all $i \in F \cup B$, $t \in T$.

6. **Integer Production for Food:**
    * $X_{it} \in \mathbb{Z}$ for all $i \in F$, $t \in T$. 

This mathematical model represents the food and beverage production planning problem. The objective is to maximize profit by determining the optimal production, storage, and sales quantities for each product in each time period while considering the various cost factors, demand limits, and capacity restrictions. 


## 3. Generate the pyomo code

In [337]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [338]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data from CSV files)
fixed_cost_production = {
    (1, 1): 100, (1, 2): 110, (1, 3): 120, (1, 4): 130, (1, 5): 140, (1, 6): 150, (1, 7): 160, (1, 8): 170, (1, 9): 180, (1, 10): 190, (1, 11): 200, (1, 12): 210,
    (2, 1): 80, (2, 2): 90, (2, 3): 100, (2, 4): 110, (2, 5): 120, (2, 6): 130, (2, 7): 140, (2, 8): 150, (2, 9): 160, (2, 10): 170, (2, 11): 180, (2, 12): 190,
    (3, 1): 90, (3, 2): 100, (3, 3): 110, (3, 4): 120, (3, 5): 130, (3, 6): 140, (3, 7): 150, (3, 8): 160, (3, 9): 170, (3, 10): 180, (3, 11): 190, (3, 12): 200,
    (4, 1): 70, (4, 2): 80, (4, 3): 90, (4, 4): 100, (4, 5): 110, (4, 6): 120, (4, 7): 130, (4, 8): 140, (4, 9): 150, (4, 10): 160, (4, 11): 170, (4, 12): 180,
    (5, 1): 100, (5, 2): 110, (5, 3): 120, (5, 4): 130, (5, 5): 140, (5, 6): 150, (5, 7): 160, (5, 8): 170, (5, 9): 180, (5, 10): 190, (5, 11): 200, (5, 12): 210
}

variable_cost_production = {
    (1, 1): 2, (1, 2): 2.2, (1, 3): 2.4, (1, 4): 2.6, (1, 5): 2.8, (1, 6): 3, (1, 7): 3.2, (1, 8): 3.4, (1, 9): 3.6, (1, 10): 3.8, (1, 11): 4, (1, 12): 4.2,
    (2, 1): 1.8, (2, 2): 2, (2, 3): 2.2, (2, 4): 2.4, (2, 5): 2.6, (2, 6): 2.8, (2, 7): 3, (2, 8): 3.2, (2, 9): 3.4, (2, 10): 3.6, (2, 11): 3.8, (2, 12): 4,
    (3, 1): 2.1, (3, 2): 2.3, (3, 3): 2.5, (3, 4): 2.7, (3, 5): 2.9, (3, 6): 3.1, (3, 7): 3.3, (3, 8): 3.5, (3, 9): 3.7, (3, 10): 3.9, (3, 11): 4.1, (3, 12): 4.3,
    (4, 1): 1.7, (4, 2): 1.9, (4, 3): 2.1, (4, 4): 2.3, (4, 5): 2.5, (4, 6): 2.7, (4, 7): 2.9, (4, 8): 3.1, (4, 9): 3.3, (4, 10): 3.5, (4, 11): 3.7, (4, 12): 3.9,
    (5, 1): 2, (5, 2): 2.2, (5, 3): 2.4, (5, 4): 2.6, (5, 5): 2.8, (5, 6): 3, (5, 7): 3.2, (5, 8): 3.4, (5, 9): 3.6, (5, 10): 3.8, (5, 11): 4, (5, 12): 4.2
}

variable_cost_storage = {
    (1, 1): 0.5, (1, 2): 0.6, (1, 3): 0.7, (1, 4): 0.8, (1, 5): 0.9, (1, 6): 1, (1, 7): 1.1, (1, 8): 1.2, (1, 9): 1.3, (1, 10): 1.4, (1, 11): 1.5, (1, 12): 1.6,
    (2, 1): 0.4, (2, 2): 0.5, (2, 3): 0.6, (2, 4): 0.7, (2, 5): 0.8, (2, 6): 0.9, (2, 7): 1, (2, 8): 1.1, (2, 9): 1.2, (2, 10): 1.3, (2, 11): 1.4, (2, 12): 1.5,
    (3, 1): 0.45, (3, 2): 0.55, (3, 3): 0.65, (3, 4): 0.75, (3, 5): 0.85, (3, 6): 0.95, (3, 7): 1.05, (3, 8): 1.15, (3, 9): 1.25, (3, 10): 1.35, (3, 11): 1.45, (3, 12): 1.55,
    (4, 1): 0.35, (4, 2): 0.45, (4, 3): 0.55, (4, 4): 0.65, (4, 5): 0.75, (4, 6): 0.85, (4, 7): 0.95, (4, 8): 1.05, (4, 9): 1.15, (4, 10): 1.25, (4, 11): 1.35, (4, 12): 1.45,
    (5, 1): 0.5, (5, 2): 0.6, (5, 3): 0.7, (5, 4): 0.8, (5, 5): 0.9, (5, 6): 1, (5, 7): 1.1, (5, 8): 1.2, (5, 9): 1.3, (5, 10): 1.4, (5, 11): 1.5, (5, 12): 1.6
}

demand = {
    (1, 1): 500, (1, 2): 550, (1, 3): 600, (1, 4): 650, (1, 5): 700, (1, 6): 750, (1, 7): 800, (1, 8): 850, (1, 9): 900, (1, 10): 950, (1, 11): 1000, (1, 12): 1050,
    (2, 1): 450, (2, 2): 500, (2, 3): 550, (2, 4): 600, (2, 5): 650, (2, 6): 700, (2, 7): 750, (2, 8): 800, (2, 9): 850, (2, 10): 900, (2, 11): 950, (2, 12): 1000,
    (3, 1): 300, (3, 2): 350, (3, 3): 400, (3, 4): 450, (3, 5): 500, (3, 6): 550, (3, 7): 600, (3, 8): 650, (3, 9): 700, (3, 10): 750, (3, 11): 800, (3, 12): 850,
    (4, 1): 400, (4, 2): 450, (4, 3): 500, (4, 4): 550, (4, 5): 600, (4, 6): 650, (4, 7): 700, (4, 8): 750, (4, 9): 800, (4, 10): 850, (4, 11): 900, (4, 12): 950,
    (5, 1): 250, (5, 2): 300, (5, 3): 350, (5, 4): 400, (5, 5): 450, (5, 6): 500, (5, 7): 550, (5, 8): 600, (5, 9): 650, (5, 10): 700, (5, 11): 750, (5, 12): 800
}

revenue = {
    (1, 1): 5, (1, 2): 5.5, (1, 3): 6, (1, 4): 6.5, (1, 5): 7, (1, 6): 7.5, (1, 7): 8, (1, 8): 8.5, (1, 9): 9, (1, 10): 9.5, (1, 11): 10, (1, 12): 10.5,
    (2, 1): 4.5, (2, 2): 5, (2, 3): 5.5, (2, 4): 6, (2, 5): 6.5, (2, 6): 7, (2, 7): 7.5, (2, 8): 8, (2, 9): 8.5, (2, 10): 9, (2, 11): 9.5, (2, 12): 10,
    (3, 1): 4, (3, 2): 4.5, (3, 3): 5, (3, 4): 5.5, (3, 5): 6, (3, 6): 6.5, (3, 7): 7, (3, 8): 7.5, (3, 9): 8, (3, 10): 8.5, (3, 11): 9, (3, 12): 9.5,
    (4, 1): 3.5, (4, 2): 4, (4, 3): 4.5, (4, 4): 5, (4, 5): 5.5, (4, 6): 6, (4, 7): 6.5, (4, 8): 7, (4, 9): 7.5, (4, 10): 8, (4, 11): 8.5, (4, 12): 9,
    (5, 1): 4, (5, 2): 4.5, (5, 3): 5, (5, 4): 5.5, (5, 5): 6, (5, 6): 6.5, (5, 7): 7, (5, 8): 7.5, (5, 9): 8, (5, 10): 8.5, (5, 11): 9, (5, 12): 9.5
}

storage_capacity = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}
production_capacity = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}

# Set Initialization
model = pyo.ConcreteModel()
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[1, 2, 3])  # Beverage products
model.FB = model.F | model.B  # All products

# Parameter Initialization
model.FC = pyo.Param(model.FB, model.T, initialize=fixed_cost_production)
model.VCprod = pyo.Param(model.FB, model.T, initialize=variable_cost_production)
model.VCstore = pyo.Param(model.FB, model.T, initialize=variable_cost_storage)
model.D = pyo.Param(model.FB, model.T, initialize=demand)
model.R = pyo.Param(model.FB, model.T, initialize=revenue)
model.SC = pyo.Param(model.FB, initialize=storage_capacity)
model.PC = pyo.Param(model.FB, initialize=production_capacity)

# Decision Variable Initialization
model.X = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity produced
model.S = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity stored
model.Y = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity sold

# Integer Production for Food
model.X_integer = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)
model.integer_constraint = pyo.Constraint(model.F, model.T, rule=lambda model, i, t: model.X[i, t] == model.X_integer[i, t])

# Objective Function
def objective_rule(model):
    return sum(model.R[i, t] * model.Y[i, t] - model.VCprod[i, t] * model.X[i, t] - model.VCstore[i, t] * model.S[i, t] - model.FC[i, t] * (model.X[i, t] > 0) for i in model.FB for t in model.T)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
model.production_capacity = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.X[i, t] <= model.PC[i])
model.storage_capacity = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.S[i, t] <= model.SC[i])

def inventory_balance_rule(model, i, t):
    if t == 1:
        return model.X[i, t] - model.Y[i, t] == model.S[i, t]
    else:
        return model.S[i, t-1] + model.X[i, t] - model.Y[i, t] == model.S[i, t]
model.inventory_balance = pyo.Constraint(model.FB, model.T, rule=inventory_balance_rule)

model.demand_satisfaction = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.Y[i, t] <= model.D[i, t])

# Solve the model
solver = pyo.SolverFactory('cbc') # You can choose a different solver
results = solver.solve(model)

# Display results
print(results)
print("Profit:", pyo.value(model.profit))
```

This code defines the Pyomo model for the food and beverage production planning problem. The sample data for the parameters is given, but in reality, you should replace it with the actual data from the respective CSV files. 

**Explanation of the code:**

1. **Import pyomo.environ as pyo:** This line imports the Pyomo library.

2. **Sample data:** Dictionaries are defined to hold the sample data. You should replace this data with your actual data.

3. **Set Initialization:** 
    * `model.T = pyo.RangeSet(1, 12)` defines the set of time periods.
    * `model.F = pyo.Set(initialize=[1, 2])` defines the set of food products.
    * `model.B = pyo.Set(initialize=[1, 2, 3])` defines the set of beverage products.
    * `model.FB = model.F | model.B` creates a set of all products (food and beverage).

4. **Parameter Initialization:** This section initializes the parameters of the model using the provided sample data.

5. **Decision Variable Initialization:** 
    *  `model.X = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals)` defines the production quantity variable.
    *  `model.S = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals)` defines the storage quantity variable.
    *  `model.Y = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals)` defines the sales quantity variable.

6. **Integer Production for Food:** This part ensures that food production quantities are integers.

7. **Objective Function:** `model.profit` is defined as the objective function to maximize total profit.

8. **Constraints:**  All the constraints outlined in the mathematical model are defined using lambda functions.

9. **Solving the Model:**  A solver is selected (in this case, 'cbc'), and the model is solved.

10. **Displaying Results:** The solver results are printed, including the optimal profit value.

Remember to replace the sample data with your actual data from the CSV files for meaningful results. 


## 4. Input problem data and try running the generated code

In [340]:
import pyomo.environ as pyo

# Sample Data (replace with actual data from CSV files)
fixed_cost_production = {
    (1, 1): 100, (1, 2): 110, (1, 3): 120, (1, 4): 130, (1, 5): 140, (1, 6): 150, (1, 7): 160, (1, 8): 170, (1, 9): 180, (1, 10): 190, (1, 11): 200, (1, 12): 210,
    (2, 1): 80, (2, 2): 90, (2, 3): 100, (2, 4): 110, (2, 5): 120, (2, 6): 130, (2, 7): 140, (2, 8): 150, (2, 9): 160, (2, 10): 170, (2, 11): 180, (2, 12): 190,
    (3, 1): 90, (3, 2): 100, (3, 3): 110, (3, 4): 120, (3, 5): 130, (3, 6): 140, (3, 7): 150, (3, 8): 160, (3, 9): 170, (3, 10): 180, (3, 11): 190, (3, 12): 200,
    (4, 1): 70, (4, 2): 80, (4, 3): 90, (4, 4): 100, (4, 5): 110, (4, 6): 120, (4, 7): 130, (4, 8): 140, (4, 9): 150, (4, 10): 160, (4, 11): 170, (4, 12): 180,
    (5, 1): 100, (5, 2): 110, (5, 3): 120, (5, 4): 130, (5, 5): 140, (5, 6): 150, (5, 7): 160, (5, 8): 170, (5, 9): 180, (5, 10): 190, (5, 11): 200, (5, 12): 210
}

variable_cost_production = {
    (1, 1): 2, (1, 2): 2.2, (1, 3): 2.4, (1, 4): 2.6, (1, 5): 2.8, (1, 6): 3, (1, 7): 3.2, (1, 8): 3.4, (1, 9): 3.6, (1, 10): 3.8, (1, 11): 4, (1, 12): 4.2,
    (2, 1): 1.8, (2, 2): 2, (2, 3): 2.2, (2, 4): 2.4, (2, 5): 2.6, (2, 6): 2.8, (2, 7): 3, (2, 8): 3.2, (2, 9): 3.4, (2, 10): 3.6, (2, 11): 3.8, (2, 12): 4,
    (3, 1): 2.1, (3, 2): 2.3, (3, 3): 2.5, (3, 4): 2.7, (3, 5): 2.9, (3, 6): 3.1, (3, 7): 3.3, (3, 8): 3.5, (3, 9): 3.7, (3, 10): 3.9, (3, 11): 4.1, (3, 12): 4.3,
    (4, 1): 1.7, (4, 2): 1.9, (4, 3): 2.1, (4, 4): 2.3, (4, 5): 2.5, (4, 6): 2.7, (4, 7): 2.9, (4, 8): 3.1, (4, 9): 3.3, (4, 10): 3.5, (4, 11): 3.7, (4, 12): 3.9,
    (5, 1): 2, (5, 2): 2.2, (5, 3): 2.4, (5, 4): 2.6, (5, 5): 2.8, (5, 6): 3, (5, 7): 3.2, (5, 8): 3.4, (5, 9): 3.6, (5, 10): 3.8, (5, 11): 4, (5, 12): 4.2
}

variable_cost_storage = {
    (1, 1): 0.5, (1, 2): 0.6, (1, 3): 0.7, (1, 4): 0.8, (1, 5): 0.9, (1, 6): 1, (1, 7): 1.1, (1, 8): 1.2, (1, 9): 1.3, (1, 10): 1.4, (1, 11): 1.5, (1, 12): 1.6,
    (2, 1): 0.4, (2, 2): 0.5, (2, 3): 0.6, (2, 4): 0.7, (2, 5): 0.8, (2, 6): 0.9, (2, 7): 1, (2, 8): 1.1, (2, 9): 1.2, (2, 10): 1.3, (2, 11): 1.4, (2, 12): 1.5,
    (3, 1): 0.45, (3, 2): 0.55, (3, 3): 0.65, (3, 4): 0.75, (3, 5): 0.85, (3, 6): 0.95, (3, 7): 1.05, (3, 8): 1.15, (3, 9): 1.25, (3, 10): 1.35, (3, 11): 1.45, (3, 12): 1.55,
    (4, 1): 0.35, (4, 2): 0.45, (4, 3): 0.55, (4, 4): 0.65, (4, 5): 0.75, (4, 6): 0.85, (4, 7): 0.95, (4, 8): 1.05, (4, 9): 1.15, (4, 10): 1.25, (4, 11): 1.35, (4, 12): 1.45,
    (5, 1): 0.5, (5, 2): 0.6, (5, 3): 0.7, (5, 4): 0.8, (5, 5): 0.9, (5, 6): 1, (5, 7): 1.1, (5, 8): 1.2, (5, 9): 1.3, (5, 10): 1.4, (5, 11): 1.5, (5, 12): 1.6
}

demand = {
    (1, 1): 500, (1, 2): 550, (1, 3): 600, (1, 4): 650, (1, 5): 700, (1, 6): 750, (1, 7): 800, (1, 8): 850, (1, 9): 900, (1, 10): 950, (1, 11): 1000, (1, 12): 1050,
    (2, 1): 450, (2, 2): 500, (2, 3): 550, (2, 4): 600, (2, 5): 650, (2, 6): 700, (2, 7): 750, (2, 8): 800, (2, 9): 850, (2, 10): 900, (2, 11): 950, (2, 12): 1000,
    (3, 1): 300, (3, 2): 350, (3, 3): 400, (3, 4): 450, (3, 5): 500, (3, 6): 550, (3, 7): 600, (3, 8): 650, (3, 9): 700, (3, 10): 750, (3, 11): 800, (3, 12): 850,
    (4, 1): 400, (4, 2): 450, (4, 3): 500, (4, 4): 550, (4, 5): 600, (4, 6): 650, (4, 7): 700, (4, 8): 750, (4, 9): 800, (4, 10): 850, (4, 11): 900, (4, 12): 950,
    (5, 1): 250, (5, 2): 300, (5, 3): 350, (5, 4): 400, (5, 5): 450, (5, 6): 500, (5, 7): 550, (5, 8): 600, (5, 9): 650, (5, 10): 700, (5, 11): 750, (5, 12): 800
}

revenue = {
    (1, 1): 5, (1, 2): 5.5, (1, 3): 6, (1, 4): 6.5, (1, 5): 7, (1, 6): 7.5, (1, 7): 8, (1, 8): 8.5, (1, 9): 9, (1, 10): 9.5, (1, 11): 10, (1, 12): 10.5,
    (2, 1): 4.5, (2, 2): 5, (2, 3): 5.5, (2, 4): 6, (2, 5): 6.5, (2, 6): 7, (2, 7): 7.5, (2, 8): 8, (2, 9): 8.5, (2, 10): 9, (2, 11): 9.5, (2, 12): 10,
    (3, 1): 4, (3, 2): 4.5, (3, 3): 5, (3, 4): 5.5, (3, 5): 6, (3, 6): 6.5, (3, 7): 7, (3, 8): 7.5, (3, 9): 8, (3, 10): 8.5, (3, 11): 9, (3, 12): 9.5,
    (4, 1): 3.5, (4, 2): 4, (4, 3): 4.5, (4, 4): 5, (4, 5): 5.5, (4, 6): 6, (4, 7): 6.5, (4, 8): 7, (4, 9): 7.5, (4, 10): 8, (4, 11): 8.5, (4, 12): 9,
    (5, 1): 4, (5, 2): 4.5, (5, 3): 5, (5, 4): 5.5, (5, 5): 6, (5, 6): 6.5, (5, 7): 7, (5, 8): 7.5, (5, 9): 8, (5, 10): 8.5, (5, 11): 9, (5, 12): 9.5
}

storage_capacity = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}
production_capacity = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}

# Set Initialization
model = pyo.ConcreteModel()
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[3, 4, 5])  # Beverage products
model.FB = model.F | model.B  # All products

# Parameter Initialization
model.FC = pyo.Param(model.FB, model.T, initialize=fixed_cost_production)
model.VCprod = pyo.Param(model.FB, model.T, initialize=variable_cost_production)
model.VCstore = pyo.Param(model.FB, model.T, initialize=variable_cost_storage)
model.D = pyo.Param(model.FB, model.T, initialize=demand)
model.R = pyo.Param(model.FB, model.T, initialize=revenue)
model.SC = pyo.Param(model.FB, initialize=storage_capacity)
model.PC = pyo.Param(model.FB, initialize=production_capacity)

# Decision Variable Initialization
model.X = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity produced
model.S = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity stored
model.Y = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity sold

# Integer Production for Food
model.X_integer = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)
model.integer_constraint = pyo.Constraint(model.F, model.T, rule=lambda model, i, t: model.X[i, t] == model.X_integer[i, t])

# Objective Function
def objective_rule(model):
    return sum(model.R[i, t] * model.Y[i, t] - model.VCprod[i, t] * model.X[i, t] - model.VCstore[i, t] * model.S[i, t] - model.FC[i, t] * (model.X[i, t] > 0) for i in model.FB for t in model.T)
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
model.production_capacity = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.X[i, t] <= model.PC[i])
model.storage_capacity = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.S[i, t] <= model.SC[i])

def inventory_balance_rule(model, i, t):
    if t == 1:
        return model.X[i, t] - model.Y[i, t] == model.S[i, t]
    else:
        return model.S[i, t-1] + model.X[i, t] - model.Y[i, t] == model.S[i, t]
model.inventory_balance = pyo.Constraint(model.FB, model.T, rule=inventory_balance_rule)

model.demand_satisfaction = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.Y[i, t] <= model.D[i, t])

# Solve the model
solver = pyo.SolverFactory('cbc') # You can choose a different solver
results = solver.solve(model)

# Display results
print(results)
print("Profit:", pyo.value(model.profit))

ERROR: Rule failed when generating expression for Objective profit with index
None: TypeError: unsupported operand type(s) for *: 'int' and
'InequalityExpression'
ERROR: Constructing component 'profit' from data=None failed:
        TypeError: unsupported operand type(s) for *: 'int' and
        'InequalityExpression'


TypeError: unsupported operand type(s) for *: 'int' and 'InequalityExpression'

## 5. Correct the code to verify model viability (optional)

In [350]:
import pyomo.environ as pyo

# # Sample Data (replace with actual data from CSV files)
# fixed_cost_production = {
#     (1, 1): 100, (1, 2): 110, (1, 3): 120, (1, 4): 130, (1, 5): 140, (1, 6): 150, (1, 7): 160, (1, 8): 170, (1, 9): 180, (1, 10): 190, (1, 11): 200, (1, 12): 210,
#     (2, 1): 80, (2, 2): 90, (2, 3): 100, (2, 4): 110, (2, 5): 120, (2, 6): 130, (2, 7): 140, (2, 8): 150, (2, 9): 160, (2, 10): 170, (2, 11): 180, (2, 12): 190,
#     (3, 1): 90, (3, 2): 100, (3, 3): 110, (3, 4): 120, (3, 5): 130, (3, 6): 140, (3, 7): 150, (3, 8): 160, (3, 9): 170, (3, 10): 180, (3, 11): 190, (3, 12): 200,
#     (4, 1): 70, (4, 2): 80, (4, 3): 90, (4, 4): 100, (4, 5): 110, (4, 6): 120, (4, 7): 130, (4, 8): 140, (4, 9): 150, (4, 10): 160, (4, 11): 170, (4, 12): 180,
#     (5, 1): 100, (5, 2): 110, (5, 3): 120, (5, 4): 130, (5, 5): 140, (5, 6): 150, (5, 7): 160, (5, 8): 170, (5, 9): 180, (5, 10): 190, (5, 11): 200, (5, 12): 210
# }

# variable_cost_production = {
#     (1, 1): 2, (1, 2): 2.2, (1, 3): 2.4, (1, 4): 2.6, (1, 5): 2.8, (1, 6): 3, (1, 7): 3.2, (1, 8): 3.4, (1, 9): 3.6, (1, 10): 3.8, (1, 11): 4, (1, 12): 4.2,
#     (2, 1): 1.8, (2, 2): 2, (2, 3): 2.2, (2, 4): 2.4, (2, 5): 2.6, (2, 6): 2.8, (2, 7): 3, (2, 8): 3.2, (2, 9): 3.4, (2, 10): 3.6, (2, 11): 3.8, (2, 12): 4,
#     (3, 1): 2.1, (3, 2): 2.3, (3, 3): 2.5, (3, 4): 2.7, (3, 5): 2.9, (3, 6): 3.1, (3, 7): 3.3, (3, 8): 3.5, (3, 9): 3.7, (3, 10): 3.9, (3, 11): 4.1, (3, 12): 4.3,
#     (4, 1): 1.7, (4, 2): 1.9, (4, 3): 2.1, (4, 4): 2.3, (4, 5): 2.5, (4, 6): 2.7, (4, 7): 2.9, (4, 8): 3.1, (4, 9): 3.3, (4, 10): 3.5, (4, 11): 3.7, (4, 12): 3.9,
#     (5, 1): 2, (5, 2): 2.2, (5, 3): 2.4, (5, 4): 2.6, (5, 5): 2.8, (5, 6): 3, (5, 7): 3.2, (5, 8): 3.4, (5, 9): 3.6, (5, 10): 3.8, (5, 11): 4, (5, 12): 4.2
# }

# variable_cost_storage = {
#     (1, 1): 0.5, (1, 2): 0.6, (1, 3): 0.7, (1, 4): 0.8, (1, 5): 0.9, (1, 6): 1, (1, 7): 1.1, (1, 8): 1.2, (1, 9): 1.3, (1, 10): 1.4, (1, 11): 1.5, (1, 12): 1.6,
#     (2, 1): 0.4, (2, 2): 0.5, (2, 3): 0.6, (2, 4): 0.7, (2, 5): 0.8, (2, 6): 0.9, (2, 7): 1, (2, 8): 1.1, (2, 9): 1.2, (2, 10): 1.3, (2, 11): 1.4, (2, 12): 1.5,
#     (3, 1): 0.45, (3, 2): 0.55, (3, 3): 0.65, (3, 4): 0.75, (3, 5): 0.85, (3, 6): 0.95, (3, 7): 1.05, (3, 8): 1.15, (3, 9): 1.25, (3, 10): 1.35, (3, 11): 1.45, (3, 12): 1.55,
#     (4, 1): 0.35, (4, 2): 0.45, (4, 3): 0.55, (4, 4): 0.65, (4, 5): 0.75, (4, 6): 0.85, (4, 7): 0.95, (4, 8): 1.05, (4, 9): 1.15, (4, 10): 1.25, (4, 11): 1.35, (4, 12): 1.45,
#     (5, 1): 0.5, (5, 2): 0.6, (5, 3): 0.7, (5, 4): 0.8, (5, 5): 0.9, (5, 6): 1, (5, 7): 1.1, (5, 8): 1.2, (5, 9): 1.3, (5, 10): 1.4, (5, 11): 1.5, (5, 12): 1.6
# }

# demand = {
#     (1, 1): 500, (1, 2): 550, (1, 3): 600, (1, 4): 650, (1, 5): 700, (1, 6): 750, (1, 7): 800, (1, 8): 850, (1, 9): 900, (1, 10): 950, (1, 11): 1000, (1, 12): 1050,
#     (2, 1): 450, (2, 2): 500, (2, 3): 550, (2, 4): 600, (2, 5): 650, (2, 6): 700, (2, 7): 750, (2, 8): 800, (2, 9): 850, (2, 10): 900, (2, 11): 950, (2, 12): 1000,
#     (3, 1): 300, (3, 2): 350, (3, 3): 400, (3, 4): 450, (3, 5): 500, (3, 6): 550, (3, 7): 600, (3, 8): 650, (3, 9): 700, (3, 10): 750, (3, 11): 800, (3, 12): 850,
#     (4, 1): 400, (4, 2): 450, (4, 3): 500, (4, 4): 550, (4, 5): 600, (4, 6): 650, (4, 7): 700, (4, 8): 750, (4, 9): 800, (4, 10): 850, (4, 11): 900, (4, 12): 950,
#     (5, 1): 250, (5, 2): 300, (5, 3): 350, (5, 4): 400, (5, 5): 450, (5, 6): 500, (5, 7): 550, (5, 8): 600, (5, 9): 650, (5, 10): 700, (5, 11): 750, (5, 12): 800
# }

# revenue = {
#     (1, 1): 5, (1, 2): 5.5, (1, 3): 6, (1, 4): 6.5, (1, 5): 7, (1, 6): 7.5, (1, 7): 8, (1, 8): 8.5, (1, 9): 9, (1, 10): 9.5, (1, 11): 10, (1, 12): 10.5,
#     (2, 1): 4.5, (2, 2): 5, (2, 3): 5.5, (2, 4): 6, (2, 5): 6.5, (2, 6): 7, (2, 7): 7.5, (2, 8): 8, (2, 9): 8.5, (2, 10): 9, (2, 11): 9.5, (2, 12): 10,
#     (3, 1): 4, (3, 2): 4.5, (3, 3): 5, (3, 4): 5.5, (3, 5): 6, (3, 6): 6.5, (3, 7): 7, (3, 8): 7.5, (3, 9): 8, (3, 10): 8.5, (3, 11): 9, (3, 12): 9.5,
#     (4, 1): 3.5, (4, 2): 4, (4, 3): 4.5, (4, 4): 5, (4, 5): 5.5, (4, 6): 6, (4, 7): 6.5, (4, 8): 7, (4, 9): 7.5, (4, 10): 8, (4, 11): 8.5, (4, 12): 9,
#     (5, 1): 4, (5, 2): 4.5, (5, 3): 5, (5, 4): 5.5, (5, 5): 6, (5, 6): 6.5, (5, 7): 7, (5, 8): 7.5, (5, 9): 8, (5, 10): 8.5, (5, 11): 9, (5, 12): 9.5
# }

#DATA LOADING BY HUMAN

fixed_cost_production = pd.read_csv("fixed_cost_production.csv")
variable_cost_production = pd.read_csv("variable_cost_production.csv")
variable_cost_storage = pd.read_csv("variable_cost_storage.csv")
demand = pd.read_csv("demand.csv")
revenue = pd.read_csv("revenue.csv")

# Transform into dictionary format
fixed_cost_production = {(i, j): fixed_cost_production.iloc[i-1, j] for i in range(1, len(fixed_cost_production)+1) for j in range(1, len(fixed_cost_production.columns))}
variable_cost_production = {(i, j): variable_cost_production.iloc[i-1, j] for i in range(1, len(variable_cost_production)+1) for j in range(1, len(variable_cost_production.columns))}
variable_cost_storage = {(i, j): variable_cost_storage.iloc[i-1, j] for i in range(1, len(variable_cost_storage)+1) for j in range(1, len(variable_cost_storage.columns))}
demand = {(i, j): demand.iloc[i-1, j] for i in range(1, len(demand)+1) for j in range(1, len(demand.columns))}
revenue = {(i, j): revenue.iloc[i-1, j] for i in range(1, len(revenue)+1) for j in range(1, len(revenue.columns))}


#END
storage_capacity = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}
production_capacity = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}

# Set Initialization
model = pyo.ConcreteModel()
model.T = pyo.RangeSet(1, 12)  # Time periods
model.F = pyo.Set(initialize=[1, 2])  # Food products
model.B = pyo.Set(initialize=[3, 4, 5])  # Beverage products
model.FB = model.F | model.B  # All products

# Parameter Initialization
model.FC = pyo.Param(model.FB, model.T, initialize=fixed_cost_production)
model.VCprod = pyo.Param(model.FB, model.T, initialize=variable_cost_production)
model.VCstore = pyo.Param(model.FB, model.T, initialize=variable_cost_storage)
model.D = pyo.Param(model.FB, model.T, initialize=demand)
model.R = pyo.Param(model.FB, model.T, initialize=revenue)
model.SC = pyo.Param(model.FB, initialize=storage_capacity)
model.PC = pyo.Param(model.FB, initialize=production_capacity)

# Decision Variable Initialization
model.X = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity produced
model.S = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity stored
model.Y = pyo.Var(model.FB, model.T, within=pyo.NonNegativeReals) # Quantity sold

# Integer Production for Food
model.X_integer = pyo.Var(model.F, model.T, within=pyo.NonNegativeIntegers)
model.integer_constraint = pyo.Constraint(model.F, model.T, rule=lambda model, i, t: model.X[i, t] == model.X_integer[i, t])

def objective_rule(model):
    return sum(
        model.R[i, t] * model.Y[i, t] 
        - model.VCprod[i, t] * model.X[i, t] 
        - model.VCstore[i, t] * model.S[i, t] 
        - model.FC[i, t] * (1 - pyo.exp(-model.X[i, t] / model.PC[i]))  # Adding fixed costs using an indicator function
        for i in model.FB 
        for t in model.T
    )
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
model.production_capacity = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.X[i, t] <= model.PC[i])
model.storage_capacity = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.S[i, t] <= model.SC[i])

def inventory_balance_rule(model, i, t):
    if t == 1:
        return model.X[i, t] - model.Y[i, t] == model.S[i, t]
    else:
        return model.S[i, t-1] + model.X[i, t] - model.Y[i, t] == model.S[i, t]
model.inventory_balance = pyo.Constraint(model.FB, model.T, rule=inventory_balance_rule)

model.demand_satisfaction = pyo.Constraint(model.FB, model.T, rule=lambda model, i, t: model.Y[i, t] <= model.D[i, t])

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver
results = solver.solve(model)

# Display results
print(results)
print("Profit:", pyo.value(model.profit))


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 264
  Number of variables: 204
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.06061959266662598
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit: 12371.505923364608


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [333]:
print(response.text)

## Mathematical Optimization Model for Production Planning

**Sets and Indices:**

*  $t \in T = \{1, 2, ..., 12\}$: Set of time periods.
*  $f \in F = \{1, 2\}$: Set of food products.
*  $b \in B = \{1, 2, 3\}$: Set of beverage products.

**Parameters:**

*  $FC_{ft}$: Fixed cost of producing one unit of food product $f$ in time period $t$ (from "fixed_cost_production.csv").
*  $VC_{ft}$: Variable cost of producing one unit of food product $f$ in time period $t$ (from "variable_cost_production.csv").
*  $VC_{bt}$: Variable cost of producing one liter of beverage product $b$ in time period $t$ (from "variable_cost_production.csv").
*  $SC_{ft}$: Variable cost of storing one unit of food product $f$ in time period $t$ (from "variable_cost_storage.csv").
*  $SC_{bt}$: Variable cost of storing one liter of beverage product $b$ in time period $t$ (from "variable_cost_storage.csv").
*  $D_{ft}$: Demand for food product $f$ in time period $t$ (from "demand.csv").
*  $D_{bt}$: Demand for beve

In [334]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {
    (1, 1): 100,
    (1, 2): 110,
    (1, 3): 105,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 120,
    (2, 2): 115,
    (2, 3): 125,
    # ... Add other data points for food product 2 in other time periods
}
variable_cost_production = {
    (1, 1): 10,
    (1, 2): 12,
    (1, 3): 11,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 13,
    (2, 2): 14,
    (2, 3): 15,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 5,  # Beverage 1
    (1, 2): 6,
    # ...
}
variable_cost_storage = {
    (1, 1): 2,
    (1, 2): 3,
    (1, 3): 2.5,
    # ... Add other data points for food product 1 in other time periods
    (2, 1): 2.5,
    (2, 2): 3.5,
    (2, 3): 3,
    # ... Add other data points for food product 2 in other time periods
    (1, 1): 1,  # Beverage 1
    (